In [ ]:
#The purpose of this is to use an exon centric analysis:
#Take all exons and label as constitutive vs spliced 
#Associate exons with iCLIP binding to get the total

In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from math import *
from subprocess import *
import pybedtools as pbt
from glob import glob
import seaborn as sns
import statsmodels.formula.api as sm
import csv
import re
csv.register_dialect("textdialect",delimiter='\t')
%matplotlib inline

/home/sam/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
#datasets to use for RT stops = mega dataframe_gene_expression_normalized_1nt
#this one has all of the information and is also normalized with gene calls to the highest expression in a given sample
clip_df = pd.read_csv('hnM_DMSO_TAM_master_dataframe_gene_expression_normalized_1nt',sep='\t')

In [3]:
clip_df.head()

,cluster_id,chrom,start,end,ens_gene,blank,strand,hnM_DMSO,hnM_TAM,hnM_DMSO_1,...,external_gene_name,biotype,twist_d0_fpkm_1,twist_d14_fpkm_1,log2ratio_fpkm_d14_d0,hnM_DMSO_clip_gene_fpkm_norm,hnM_TAM_clip_gene_fpkm_norm,hnM_DMSO_clip_gene_fpkm_norm_1,hnM_TAM_clip_gene_fpkm_norm_1,log2ratio_hnM_TAM_DMSO_clip_gene_fpkm_norm
0,chr1:251639-251640,chr1,251639,251640,ENSG00000228463,0,-,6.521757,0.000000,7.521757,...,AP006222.2,lincRNA,1.040763,1.117389,0.102490,6.266323,0.000000,7.266323,1.000000,-2.861225
1,chr1:251646-251647,chr1,251646,251647,ENSG00000228463,0,-,2.898558,0.000000,3.898558,...,AP006222.2,lincRNA,1.040763,1.117389,0.102490,2.785032,0.000000,3.785032,1.000000,-1.920306
2,chr1:564514-564515,chr1,564514,564515,ENSG00000225972,0,+,0.000000,5.691002,1.000000,...,MTND1P23,unprocessed_pseudogene,9.666940,27.279600,1.496691,0.000000,0.208618,1.000000,1.208618,0.273358
3,chr1:564595-564596,chr1,564595,564596,ENSG00000225972,0,+,5.072477,0.000000,6.072477,...,MTND1P23,unprocessed_pseudogene,9.666940,27.279600,1.496691,0.524724,0.000000,1.524724,1.000000,-0.608548
4,chr1:564722-564723,chr1,564722,564723,ENSG00000225972,0,+,0.000000,19.349407,1.000000,...,MTND1P23,unprocessed_pseudogene,9.666940,27.279600,1.496691,0.000000,0.709300,1.000000,1.709300,0.773405


In [4]:
clip_df.columns.tolist()

['cluster_id',
 'chrom',
 'start',
 'end',
 'ens_gene',
 'blank',
 'strand',
 'hnM_DMSO',
 'hnM_TAM',
 'hnM_DMSO_1',
 'hnM_TAM_1',
 'log2ratio_TAM-DMSO',
 'gene',
 'gene_coords',
 'twist_d0_fpkm',
 'twist_d14_fpkm',
 'twist_d0_d14_fpkm_sum',
 'twist_d0_tpm',
 'twist_d14_tpm',
 'chr',
 'coords',
 'gene_length',
 'external_gene_name',
 'biotype',
 'twist_d0_fpkm_1',
 'twist_d14_fpkm_1',
 'log2ratio_fpkm_d14_d0',
 'hnM_DMSO_clip_gene_fpkm_norm',
 'hnM_TAM_clip_gene_fpkm_norm',
 'hnM_DMSO_clip_gene_fpkm_norm_1',
 'hnM_TAM_clip_gene_fpkm_norm_1',
 'log2ratio_hnM_TAM_DMSO_clip_gene_fpkm_norm']

In [ ]:
#Splicing library
#Description of ucsc known alt
# This track shows various types of alternative splicing and other events that result in more than a single transcript from the same gene. The label by an item describes the type of event. The events are:

# Alternate Promoter (altPromoter) - Transcription starts at multiple places. The altPromoter extends from 100 bases before to 50 bases after transcription start.
# Alternate Finish Site (altFinish) - Transcription ends at multiple places.
# Cassette Exon (cassetteExon) - Exon is present in some transcripts but not others. These are found by looking for exons that overlap an intron in the same transcript.
# Retained Intron (retainedIntron) - Introns are spliced out in some transcripts but not others. In some cases, particularly when the intron is near the 3' end, this can reflect an incompletely processed transcript rather than a true alt-splicing event.
# Overlapping Exon (bleedingExon) - Initial or terminal exons overlap in an intron in another transcript. These often are associated with incompletely processed transcripts.
# Alternate 3' End (altThreePrime) - Variations on the 3' end of an intron.
# Alternate 5' End (altFivePrime) - Variations on the 5' end of an intron.
# Intron Ends have AT/AC (atacIntron) - An intron with AT/AC ends rather than the usual GT/AG. These are associated with the minor spliceosome.
# Strange Intron Ends (strangeSplice) - An intron with ends that are not GT/AG, GC/AG, or AT/AC. These are usually artifacts of some sort due to sequencing error or polymorphism

#rmats splicing library
def getASEvents(asDir,): ## get AS events from GTF and SAM files
  logging.debug("getting AS events function..");

  samNames1=uniqSamNames(1);
  samNames2=uniqSamNames(2);
  finalSamNames = ','.join(samNames1)+','+','.join(samNames2);  
  if lite==1:
    finalSamNames = ','.join(samNames1); ## for the lite version

  #cmd = 'python '+binPath+'/processGTF.SAMs.py '+gtf+' '+asDir+'/fromGTF'+' '+finalSamNames+' '+libType+' '+tempPath;
  cmd = 'python '+binPath+'/processGTF.BAMs.py '+gtf+' '+asDir+'/fromGTF'+' '+finalSamNames+' '+libType+' '+SEPE+' '+str(novelSS) +' '+tempPath;
  #oFile.write('###### getting AS events from GTF and SAM files #####\n'+cmd+'\n#\n');
  oFile.write('###### getting AS events from GTF and BAM files #####\n'+cmd+'\n#\n');
  oFile.flush();
  status,output=commands.getstatusoutput(cmd);
  logging.debug("getting AS events is done with status %s" % status);
  if (int(status)!=0): ## it did not go well
    logging.debug("error in getting AS events %s" % status);
    logging.debug("error detail: %s" % output);
    raise Exception();
  logging.debug(output);

In [ ]:
#collect a dataframe with only the information we need for each sample
dmso_names = ['chrom','start','end','ens_gene','hnM_DMSO_clip_gene_fpkm_norm']
dmso = 

In [28]:
#add an id to each exon
with open('spliced_exon_analysis/hg19_v19_exons.bed') as f:
    counter = 1
    outlist = []
    for line in f:
        line = line.strip().split('\t')
        line[4] = "exon_"+str(counter)
        counter += 1
        outlist.append(line)
with open('spliced_exon_analysis/hg19_v19_exons_with_exon_id.bed','w') as f:
    writer = csv.writer(f,'textdialect')
    writer.writerows(outlist)

In [81]:
#first step collect exons and associate with features
hg19_v19_exons = pd.read_csv('spliced_exon_analysis/hg19_v19_exons_with_exon_id.bed',sep='\t',names=['chrom','start','stop','ens_gene','exon_id','strand'])
#remove gene name stuff after ens_gene
hg19_v19_exons['ens_gene']=hg19_v19_exons.ens_gene.str.split('.').str[0]    
#read in gene names and biotypes
#I got this from ensembl biomart using the last GRCh37 archive - ENS version 75
hg19_grch37_gene_names_and_biotype = pd.read_csv('spliced_exon_analysis/ensembl_ens_gene_gene_name_biotype_grch37_p13',sep='\t',names=['ens_gene','external_gene_name','biotype'])
#why are there duplicates?
hg19_grch37_gene_names_and_biotype.drop_duplicates(inplace=True)

#merge the exons with the gene names and biotypes
hg19_v19_exons = pd.merge(hg19_v19_exons,hg19_grch37_gene_names_and_biotype,on='ens_gene',how='left')
hg19_v19_exons.to_csv('spliced_exon_analysis/hg19_v19_exon_gene_names_and_biotypes',sep='\t',header=False,index=False,na_rep='NaN')

#create a bedtools
hg19_v19_exons_bt = pbt.BedTool.from_dataframe(hg19_v19_exons,na_rep='NaN')

#now I need to bring in the library of splice exons
#Lets try multiple sources - first use hg38 known alt
#first - hg38 known alt but lifted to hg19 - this has many more events - roughly double the number of cassette exons
known_alt = pd.read_csv('/media/sam/Data1/annotations/splicing_libraries/knownAlt_hg38_to_hg19.bed',sep='\t',names=['chrom','start','end','splicing_type','score','strand'])
known_alt_bt = pbt.BedTool.from_dataframe(known_alt)
known_alt_cassette = known_alt[known_alt.splicing_type == "cassetteExon"]
known_alt_cassette_bt = pbt.BedTool.from_dataframe(known_alt_cassette).sort()

#intersect all of known alt with the hg19_v19_exons to find those that don't intersect with any known splicing event and are thus constitutive. also collect those that are alternative
hg19_v19_constitutive_exons = hg19_v19_exons_bt.intersect(b=known_alt_bt,s=True,v=True,wa=True)
hg19_v19_alternative_exons = hg19_v19_exons_bt.intersect(b=known_alt_bt,s=True,u=True,wa=True)
print((float(len(hg19_v19_alternative_exons))/float(len(hg19_v19_exons_bt))*100),'percent of exons are alternative')

#now figure out which alternative exons are cassette exons. intersect alternative with cassette 
#should i require a fraction of overlap? 
#should I require there be a perfect match or a partial match?
hg19_v19_alternative_cassette_exons = hg19_v19_alternative_exons.intersect(b=known_alt_cassette_bt,wa=True,s=True,u=True,f=1.0,r=True)
hg19_v19_alternative_non_cassette_exons = hg19_v19_alternative_exons.intersect(b=known_alt_cassette_bt,wa=True,s=True,v=True,f=1.0,r=True)

#focus on cassette exons to start
#in the dataframe label exons as alternative or not alternative and then cassette or non-cassette
# hg19_v19_alternative_exons.to_dataframe


(38.18791573937649, 'percent of exons are alternative')


In [75]:
#now I need to bring in the library of splice exons
#Lets try multiple sources - first use hg38 known alt
#first - hg38 known alt but lifted to hg19 - this has many more events - roughly double the number of cassette exons
known_alt = pd.read_csv('/media/sam/Data1/annotations/splicing_libraries/knownAlt_hg38_to_hg19.bed',sep='\t',names=['chrom','start','end','splicing_type','score','strand'])
known_alt_bt = pbt.BedTool.from_dataframe(known_alt)
known_alt_cassette = known_alt[known_alt.splicing_type == "cassetteExon"]
known_alt_cassette_bt = pbt.BedTool.from_dataframe(known_alt_cassette).sort()

#intersect all of known alt with the hg19_v19_exons to find those that don't intersect with any known splicing event and are thus constitutive. also collect those that are alternative
hg19_v19_constitutive_exons = hg19_v19_exons_bt.intersect(b=known_alt_bt,s=True,v=True,wa=True)
hg19_v19_alternative_exons = hg19_v19_exons_bt.intersect(b=known_alt_bt,s=True,u=True,wa=True)
print((float(len(hg19_v19_alternative_exons))/float(len(hg19_v19_exons_bt))*100),'percent of exons are alternative')

#now figure out which alternative exons are cassette exons. intersect alternative with cassette 
#should i require a fraction of overlap? 
#should I require there be a perfect match or a partial match?
hg19_v19_alternative_cassette_exons = hg19_v19_alternative_exons.intersect(b=known_alt_cassette_bt,wa=True,s=True,u=True,f=1.0,r=True)
hg19_v19_alternative_non_cassette_exons = hg19_v19_alternative_exons.intersect(b=known_alt_cassette_bt,wa=True,s=True,v=True,f=1.0,r=True)

#focus on cassette exons to start
#in the dataframe label exons as alternative or not alternative and then cassette or non-cassette
# hg19_v19_alternative_exons.to_dataframe


In [83]:
print(len(hg19_v19_alternative_cassette_exons)+len(hg19_v19_alternative_non_cassette_exons))
print(len(hg19_v19_alternative_exons))

123890
123890


In [63]:
splicing_type_group = known_alt.groupby(by='splicing_type')
splicing_type_group.count()
print()

,chrom,start,end,score,strand
splicing_type,,,,,
altFinish,16198,16198,16198,16198,16198
altFivePrime,8305,8305,8305,8305,8305
altPromoter,68284,68284,68284,68284,68284
altThreePrime,14129,14129,14129,14129,14129
atacIntron,276,276,276,276,276
bleedingExon,41343,41343,41343,41343,41343
cassetteExon,64926,64926,64926,64926,64926
retainedIntron,24924,24924,24924,24924,24924
strangeSplice,6130,6130,6130,6130,6130


In [79]:
known_alt.describe()

,start,end,score
count,2.445150e+05,2.445150e+05,244515.0
mean,7.175108e+07,7.175145e+07,0.0
std,5.560061e+07,5.560064e+07,0.0
min,9.110000e+02,3.048000e+03,0.0
25%,3.125139e+07,3.125145e+07,0.0
50%,5.533152e+07,5.533339e+07,0.0
75%,1.052106e+08,1.052108e+08,0.0
max,2.492005e+08,2.492006e+08,0.0


In [80]:
hg19_v19_exons.describe()

,start,stop
count,3.244220e+05,3.244220e+05
mean,7.513365e+07,7.513404e+07
std,5.599167e+07,5.599167e+07
min,5.770000e+02,6.470000e+02
25%,3.136982e+07,3.136999e+07
50%,6.143854e+07,6.143943e+07
75%,1.113556e+08,1.113557e+08
max,2.492308e+08,2.492312e+08


In [24]:
known_alt_cassette_bt.head()

chr1	120720	120932	cassetteExon	0	-
 chr1	120873	120932	cassetteExon	0	-
 chr1	231300	231385	cassetteExon	0	-
 chr1	543334	543436	cassetteExon	0	-
 chr1	563340	563603	cassetteExon	0	-
 chr1	634306	634339	cassetteExon	0	-
 chr1	639064	639169	cassetteExon	0	-
 chr1	647090	647302	cassetteExon	0	-
 chr1	647243	647302	cassetteExon	0	-
 chr1	655411	655580	cassetteExon	0	-
 

In [ ]:
#Old way I tried this


#NOTE - THIS IS NOT THE BEST WAY TO WORK ON THIS... BETTER TO DO AN EXON CENTRIC OR SPLICING EVENT CENTRIC ANALYSIS

#NOW FIGURE OUT HOW TO BRING IN THE ALTERNATIVE EXONS
#put this in the directory spliced_exon_analysis

#Goal - associate the clusters with nearest exon using bedtools closest

#Ideal setup - have each cluster associated with nearest 1 exon. This exon must be able to be labeled as constitutive vs. alternative. This exon must then be able to be associated with a PSI value from each of our RNA seq analyses... How can I do this? Many of the splicing events will not match exactly.

#Associate clusters with all exons first. Mark whether the exon is constitutive or variable.

#HOW CAN I KNOW WHETHER AN EXON IS CONSTITUTIVE OR ALTERNATIVE?
#WHY NOT USE RMATS FOR TWIST-ER CELLS? All SE called by rMATS will be merged and take the longest one. If an exon overlaps with this list, it is alternative. If it does not, it is constitutive
#Caveat with this - We don't want false positives. maxPSI > 15%, minPSI < 85% ensures we don't take all skipped or all included exons.
#Another source of information - MISO splicing library
#Another source = hg19 knownAlt
#If the exon is going to be paired with a PSI value it must be alternative

#Then associate clusters with alternative exons from rMATS. Presumably rMATS exons are all alternatively spliced and contain no constitutively spliced exons. Can separate significantly differentially spliced from control using Carsten's publication "control alternative" cutoff (FDR > 50%, maxPSI of > 15%, minPSI < 85%)

#As an exon annotation, I am using the hg19_v19 exons from CLIPPER, this is also copied in this directory
#Other exon annotations that are available:
#FAST-iCLIP-docs = hg19_transcriptome_collapse_exon.bed
#Extract transcript regions script
hg19_v19_exons = pd.read_csv('spliced_exon_analysis/hg19_v19_exons.bed',sep='\t',names=['chrom','start','stop','ens_gene','blank','strand'])
hg19_v19_exons['ens_gene']=hg19_v19_exons.ens_gene.str.split('.').str[0]
#read in gene names and biotypes
#I got this from ensembl biomart using the last GRCh37 archive - ENS version 75
hg19_grch37_gene_names_and_biotype = pd.read_csv('spliced_exon_analysis/ensembl_ens_gene_gene_name_biotype_grch37_p13',sep='\t',names=['ens_gene','external_gene_name','biotype'])
#why are there duplicates?
hg19_grch37_gene_names_and_biotype.drop_duplicates(inplace=True)

#merge the exons with the gene names and biotypes
hg19_v19_exons = pd.merge(hg19_v19_exons,hg19_grch37_gene_names_and_biotype,on='ens_gene',how='left')
hg19_v19_exons.to_csv('spliced_exon_analysis/hg19_v19_exon_gene_names_and_biotypes',sep='\t',header=False,index=False,na_rep='NaN')

#save a copy of the merge_df clusters as a bedtools
cluster_column_list = ['chrom','start','end','ens_gene','cluster_id','strand']
merge_df.to_csv('spliced_exon_analysis/merge_df.bed',sep='\t',na_rep='NaN',columns=cluster_column_list,header=False,index=False)

#run closest bed with exons and keep the distance as a column
#handle ties by taking both items
#This means that some of the clusters will be associated with multiple exons
!bedtools closest -a spliced_exon_analysis/merge_df.bed -b spliced_exon_analysis/hg19_v19_exon_gene_names_and_biotypes -D "ref" -s -t "all" > spliced_exon_analysis/merge_df_closest_exons

!bedtools closest -a spliced_exon_analysis/merge_df.bed -b spliced_exon_analysis/hg19_v19_exon_gene_names_and_biotypes -D "ref" -s -t "first" > spliced_exon_analysis/merge_df_closest_exons_first_ties

#bring in the data from TwistER, CC, and HH rMATS into splicing analysis folder
#I used parse_rMATS_SE -fdr 0.05 -psi 0.1 -read 20
#NOTE - it may be better for me in the future to just read in the data and parse it myself however I'd like - do this later

#Should I do bedtools closest on each cluster next to an exon from CC, HH, TwistER? Set a limit of maximal distance

#bring in the data from TwistER, CC, and HH rMATS into splicing analysis folder
#I used parse_rMATS_SE -fdr 0.05 -psi 0.1 -read 20
#NOTE - it may be better for me in the future to just read in the data and parse it myself however I'd like - do this later
#Should I do bedtools closest on each cluster next to an exon from CC, HH, TwistER? Set a limit of maximal distance
cc1_cc2_rMATS_df = pd.read_csv('spliced_exon_analysis/CC-1_CC-2_rMATS_3_2_5_SE_ROT_FDR_0.05_dPSI_0.1_read_cutoff_20.txt',sep='\t')
cc3_cc4_rMATS_df = pd.read_csv('spliced_exon_analysis/CC-3_CC-4_rMATS_3_2_5_SE_ROT_FDR_0.05_dPSI_0.1_read_cutoff_20.txt',sep='\t')
hh5_hh6_rMATS_df = pd.read_csv('spliced_exon_analysis/HH-5_HH-6_rMATS_3_2_5_SE_ROT_FDR_0.05_dPSI_0.1_read_cutoff_20.txt',sep='\t')
twist_d0_d14_rMATS_df = pd.read_csv('spliced_exon_analysis/SEH-1_SEH-2_rMATS_3_2_5_SE_ROT_FDR_0.05_dPSI_0.1_read_cutoff_20.txt',sep='\t')

#Now that I have this, I want to do bedtools closest to see if this this good